In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch

c:\Users\saket.singh1\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [3]:
def load_and_process_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    dialogues = [line.strip().split('|||') for line in lines if line.strip()]
    # Process each dialogue pair
    processed_examples = []
    for dialogue in dialogues:
        customer, agent = dialogue
        combined_text = customer + tokenizer.eos_token + agent + tokenizer.eos_token
        tokenized = tokenizer(combined_text, truncation=True, max_length=512, padding='max_length')
        # Ensure the 'labels' are correctly set for loss computation
        tokenized['labels'] = tokenized['input_ids'].copy()
        processed_examples.append(tokenized)

    # Creating a DataFrame
    df = pd.DataFrame(processed_examples)

    train_df, eval_df = train_test_split(df, test_size=0.1)
    train_dataset = Dataset.from_pandas(train_df)
    eval_dataset = Dataset.from_pandas(eval_df)

    return train_dataset, eval_dataset

In [4]:
# Custom collate function to ensure dynamic padding
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_ids_padded = pad_sequence([torch.tensor(seq) for seq in input_ids], 
                                    batch_first=True, padding_value=tokenizer.pad_token_id)
    labels_padded = pad_sequence([torch.tensor(seq) for seq in labels], 
                                 batch_first=True, padding_value=tokenizer.pad_token_id)

    return {'input_ids': input_ids_padded, 'labels': labels_padded}


In [9]:
# Load and process the dataset
train_dataset, eval_dataset = load_and_process_data('../../resources/next_sentence_prediction_gpt2/nsp_fine_tuning_dataset.txt')


In [11]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=custom_collate_fn,  # Use the custom collate function
)


In [12]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "gpt2_nsp_trainer.ipynb"


In [13]:
import wandb
wandb.login(key="a9433891c8b370b61ad36f3b5c379d41d79ff4dd")


wandb: Currently logged in as: saket-singh1 (mphasis). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\saket.singh1\.netrc


True

In [14]:
# Train the model
trainer.train()

  2%|▏         | 10/540 [01:58<1:45:41, 11.96s/it]

{'loss': 12.0575, 'grad_norm': 237.56495666503906, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.06}


  4%|▎         | 20/540 [04:35<2:16:47, 15.78s/it]

{'loss': 8.9151, 'grad_norm': 238.3504638671875, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.11}


  6%|▌         | 30/540 [07:09<2:08:32, 15.12s/it]

...

100%|██████████| 540/540 [1:48:35<00:00, 10.92s/it]

{'loss': 0.0636, 'grad_norm': 1.2164207696914673, 'learning_rate': 0.0, 'epoch': 3.0}


                                                   
100%|██████████| 540/540 [1:49:57<00:00, 12.22s/it]

{'eval_loss': 0.06193813681602478, 'eval_runtime': 81.9981, 'eval_samples_per_second': 0.488, 'eval_steps_per_second': 0.244, 'epoch': 3.0}
{'train_runtime': 6600.2927, 'train_samples_per_second': 0.164, 'train_steps_per_second': 0.082, 'train_loss': 0.5489876355285997, 'epoch': 3.0}


TrainOutput(global_step=540, training_loss=0.5489876355285997, metrics={'train_runtime': 6600.2927, 'train_samples_per_second': 0.164, 'train_steps_per_second': 0.082, 'train_loss': 0.5489876355285997, 'epoch': 3.0})

In [16]:
# Save the fine-tuned model
model_path = '../../resources/next_sentence_prediction_gpt2/gpt2_finetuned_nsp'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('../../resources/next_sentence_prediction_gpt2/gpt2_finetuned\\tokenizer_config.json',
 '../../resources/next_sentence_prediction_gpt2/gpt2_finetuned\\special_tokens_map.json',
 '../../resources/next_sentence_prediction_gpt2/gpt2_finetuned\\vocab.json',
 '../../resources/next_sentence_prediction_gpt2/gpt2_finetuned\\merges.txt',
 '../../resources/next_sentence_prediction_gpt2/gpt2_finetuned\\added_tokens.json')